In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
from time import perf_counter as time

# majikoi use lanczos to downscale
# whirlpool use linear to upscale

t = time()

img1 = cv.imread('c:\\Users\\khoi\\Videos\\Screenshot 2024-09-09 00-16-18.png',0)
img2 = cv.imread('e:\\Dataset\\Azarashi\\AmakanoSSPlus\\Stand\\M\\stkana03_f.png',0)
img1 = cv.imread('r:\\HG020400_01_00.png',0)
img2 = cv.imread('r:\\HG020400_01_00.png',0)
img1 = cv.imread('c:\\Users\\khoi\\Videos\\Screenshot 2024-10-25 22-53-36.png',0)
img2 = cv.imread('r:\\HG020400_01_00.png',0)


# Initiate SIFT detector
sift = cv.SIFT_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)
print(time()-t)
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)
flann = cv.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1,des2,k=2)
# store all the good matches as per Lowe's ratio test.
good = []
for m,n in matches:
    if m.distance < 0.7*n.distance:
        good.append(m)
MIN_MATCH_COUNT = 7
if len(good)>MIN_MATCH_COUNT:
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
    M, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC,5.0)
    matchesMask = mask.ravel().tolist()
    h,w = img1.shape
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv.perspectiveTransform(pts,M)
    img2 = cv.polylines(img2,[np.int32(dst)],True,255,3, cv.LINE_AA)
else:
    print( "Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT) )
    matchesMask = None
print(time()-t)
draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)
img3 = cv.drawMatches(img1,kp1,img2,kp2,good,None,**draw_params)
plt.imshow(img3),plt.show()
print(len(good))
print(dst)
print(pts)

In [4]:
a0, a1 = dst[0][0]
b0, b1 = dst[1][0]
c0, c1 = dst[2][0]
d0, d1 = dst[3][0]

print((b1 - a1)/pts[2][0][1])
print((c1 - d1)/pts[2][0][1])
print((b1 - d1)/pts[2][0][1])
print((c1 - a1)/pts[2][0][1])
print((c0 - a0)/pts[2][0][0])
print((d0 - b0)/pts[2][0][0])
print((c0 - b0)/pts[2][0][0])
print((d0 - a0)/pts[2][0][0])

print(pts[2][0][1]/(b1 - a1))
print(pts[2][0][1]/(c1 - d1))
print(pts[2][0][1]/(b1 - d1))
print(pts[2][0][1]/(c1 - a1))
print(pts[2][0][0]/(c0 - a0))
print(pts[2][0][0]/(d0 - b0))
print(pts[2][0][0]/(c0 - b0))
print(pts[2][0][0]/(d0 - a0))

1.001094
1.0013149
1.001593
1.000816
1.0006201
1.0003326
1.000048
1.0009046
0.99890715
0.9986868
0.9984095
0.99918467
0.9993803
0.9996675
0.9999519
0.9990963


In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
from time import perf_counter as time

t = time()
img1 = cv.imread('r:/decode/ev_coc03k.png',0)          # queryImage
img2 = cv.imread('r:/decode/ev_coc03a.png',0)          # trainImage

# Initiate ORB detector
orb = cv.ORB_create()

# find the keypoints and descriptors with ORB
kp1, des1 = orb.detectAndCompute(img1,None)
kp2, des2 = orb.detectAndCompute(img2,None)
print(time()-t)


bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1,des2)
good = sorted(matches, key = lambda x:x.distance)

MIN_MATCH_COUNT = 10
if len(good)>MIN_MATCH_COUNT:
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
    M, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC,5.0)
    matchesMask = mask.ravel().tolist()
    h,w = img1.shape
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv.perspectiveTransform(pts,M)
    img2 = cv.polylines(img2,[np.int32(dst)],True,255,3, cv.LINE_AA)
else:
    print( "Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT) )
    matchesMask = None
print(time()-t)
draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)
img3 = cv.drawMatches(img1,kp1,img2,kp2,good,None,**draw_params)
plt.imshow(img3),plt.show()
print(len(good))
print(dst)
print(pts)